The idea of coupled recursion is combining multiple recursion processes together in a way they can use each others recursion calls to reduce variance. This is path reuse like. In the example of period $2$ the next point position was independent of the current one (always $\text{Uniform(0,1)}$). Coupled recursion in that case is similar to splitting but instead of making new recursion calls you reuse the recursion calls from the coupled processes. <br>

This works well when working with only $1$ Green function. The implementation of coupled processes can be  thought of a "recursion" vector where each recursion all recursive calls get shared.

